# 基于RNN的花卉识别练习

导入需要的包

In [1]:
# “skimage” python 图像处理模块，如果用的conda的虚拟环境，需要用conda install scikit-image 进行安装
# glob 文件路径查找模块
from skimage import io,transform
import glob 
import os
import tensorflow as tf
import numpy as np

数据集路径---根据自己的路径进行改写

In [2]:
path='/home/liuyanfang/lyf/Codes/GAN/flower_photos/'

由于我们的数据集图片大小不一致，所以需要resize成统一大小 这里resize成100x100x3

In [3]:
w=40
h=40
c=3


读取数据集图片并添加标签,最后的形式是data 对应图片, label 是标签，roses 0,daisy 1,sunflowers 2,tulips 3,dandelion 4.

In [4]:
def read_img(path):
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)]#逐个打开文件夹，并提取图片
    imgs=[]
    labels=[]
    for idx,folder in enumerate(cate):#用于在for循环中得到计数
        for im in glob.glob(folder+'/*.jpg'):#返回所有匹配的文件路径列表
            #print('reading the images:%s'%(im))
            img=io.imread(im)
            img=transform.resize(img,(w,h))
            imgs.append(img)
            labels.append(idx)
                  
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)#变成矩阵格式
data,label=read_img(path)

/home/yuyue/anaconda2/envs/tensorflow-gpu/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
data = np.reshape(data,(3029,w*h*c))


将数据集打乱顺序，下阶段课程会讲其他方法

In [6]:
num_example=data.shape[0] # data.shape是(3029, 100, 100, 3)
arr=np.arange(num_example)# 创建等差数组 0，1，...,3028
np.random.shuffle(arr)# 打乱顺序
data=data[arr]
label=label[arr]
print(label)


[4 3 4 ... 2 4 3]


将标签onehot

In [7]:
def to_one_hot(labels):
    l = len(labels)
    res = np.zeros((l, 5), dtype=np.float32)
    for i in range(l):
        res[i][labels[i]] = 1.
    return res
label_oh = to_one_hot(label)


将所有数据集分为训练集80%、验证集10%和测试集10%

In [8]:
ratio=0.8
s=np.int(num_example*ratio)
x_train=data[:s]
y_train=label_oh[:s]
x_val=data[s:]
y_val=label_oh[s:]

生成batch

In [9]:
def gen_batch(dataset, labelset, batchsize):
    for i in range(dataset.shape[0]//batchsize):
        pos = i * batchsize
        x = dataset[pos:pos + batchsize]
        y = labelset[pos:pos + batchsize]
        yield x,y
    remain = np.shape(dataset)[0] % batchsize
    if remain != 0:
        x = dataset[-remain:]
        y = labelset[-remain:]
        yield x,y

构建网络

In [10]:
sess = tf.InteractiveSession()

In [11]:

x_=tf.placeholder(tf.float32,shape=[None,w*h*c],name='X')
y_=tf.placeholder(tf.float32,shape=[None,5],name='y_')

构建RNN网络

In [12]:
def lstm_cell(lstm_size):
  return tf.contrib.rnn.BasicLSTMCell(lstm_size)

In [13]:
stacked_lstm = tf.contrib.rnn.MultiRNNCell(
    [lstm_cell(w*h*c)] )# 参数是： cells, state_is_tuple=True

In [14]:
state = stacked_lstm.zero_state(batch_size=tf.shape(x_)[0], dtype=tf.float32)

In [15]:
output = x_
for i in range(6):
    output, state = stacked_lstm(output, state)
print(output.shape)


(?, 4800)


In [16]:
y_rnn = tf.layers.dense(output, 5,
                    kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                    bias_initializer=tf.constant_initializer(0.1),
                    name='rnn_output_layer'
                    )

In [17]:
epoch=100
batch_size=64
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_rnn))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_rnn, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("aaaaa")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step=0
    for epc in range(epoch):      
        for x, y in gen_batch(x_train, y_train, batch_size):
            l,ac,_ = sess.run([cross_entropy, accuracy, train_step], feed_dict={x_:x,y_:y})
            if step % 10 == 0:
                print("Step: {:>4}, Loss: {:.4f}, Acc: {:.4%}".format(step, l, ac))
            step += 1
    print("Training finished.")
    l, ac= sess.run([cross_entropy, accuracy],
                                       {x_: x_val, y_: y_val})
    print("Testing Loss: {:.4f}, Testing Acc: {:.4%}".format(l, ac))
   

aaaaa
Step:    0, Loss: 1.6479, Acc: 12.5000%
Step:   10, Loss: 1.3234, Acc: 39.0625%
Step:   20, Loss: 1.2795, Acc: 34.3750%
Step:   30, Loss: 1.1465, Acc: 48.4375%
Step:   40, Loss: 1.0601, Acc: 50.0000%
Step:   50, Loss: 0.9567, Acc: 60.9375%
Step:   60, Loss: 1.0568, Acc: 53.1250%
Step:   70, Loss: 0.8874, Acc: 70.3125%
Step:   80, Loss: 0.9303, Acc: 59.3750%
Step:   90, Loss: 0.8806, Acc: 62.5000%
Step:  100, Loss: 1.0058, Acc: 51.5625%
Step:  110, Loss: 0.9805, Acc: 53.1250%
Step:  120, Loss: 0.8658, Acc: 60.9375%
Step:  130, Loss: 0.9183, Acc: 53.1250%
Step:  140, Loss: 0.9808, Acc: 59.3750%
Step:  150, Loss: 1.0006, Acc: 56.2500%
Step:  160, Loss: 0.7216, Acc: 76.5625%
Step:  170, Loss: 0.8010, Acc: 67.1875%
Step:  180, Loss: 1.0488, Acc: 57.8125%
Step:  190, Loss: 0.9947, Acc: 53.1250%
Step:  200, Loss: 0.8703, Acc: 67.1875%
Step:  210, Loss: 0.7879, Acc: 68.7500%
Step:  220, Loss: 0.6907, Acc: 75.0000%
Step:  230, Loss: 0.7386, Acc: 75.0000%
Step:  240, Loss: 0.7304, Acc: 67.

Step: 2050, Loss: 0.4930, Acc: 81.2500%
Step: 2060, Loss: 0.1753, Acc: 93.7500%
Step: 2070, Loss: 0.0648, Acc: 96.8750%
Step: 2080, Loss: 0.0625, Acc: 96.8750%
Step: 2090, Loss: 0.0637, Acc: 96.8750%
Step: 2100, Loss: 0.0308, Acc: 100.0000%
Step: 2110, Loss: 0.1617, Acc: 93.7500%
Step: 2120, Loss: 0.1176, Acc: 95.3125%
Step: 2130, Loss: 0.2289, Acc: 87.5000%
Step: 2140, Loss: 0.0723, Acc: 96.8750%
Step: 2150, Loss: 0.0403, Acc: 98.4375%
Step: 2160, Loss: 0.0189, Acc: 100.0000%
Step: 2170, Loss: 0.0209, Acc: 100.0000%
Step: 2180, Loss: 0.0995, Acc: 96.8750%
Step: 2190, Loss: 0.1986, Acc: 92.1875%
Step: 2200, Loss: 0.0269, Acc: 100.0000%
Step: 2210, Loss: 0.0507, Acc: 98.4375%
Step: 2220, Loss: 0.1094, Acc: 96.8750%
Step: 2230, Loss: 0.3065, Acc: 92.1875%
Step: 2240, Loss: 0.2187, Acc: 90.6250%
Step: 2250, Loss: 0.2156, Acc: 89.0625%
Step: 2260, Loss: 0.0367, Acc: 98.4375%
Step: 2270, Loss: 0.1888, Acc: 92.1875%
Step: 2280, Loss: 0.1431, Acc: 93.7500%
Step: 2290, Loss: 0.0968, Acc: 96.87

# 结论 
## RNN可以用来做图像的识别，但毕竟图像是静态的，RNN是序列性的，所以一般不用RNN做图像识别。

训练和测试数据